### Add pointer in first cell

In [10]:
import os
import nbformat
from probml_utils.url_utils import (
    colab_to_githubraw_url,
    make_url_from_chapter_no_and_script_name,
    is_dead_url,
    colab_url_to_github_url,
)
from glob import glob

%config Completer.use_jedi = False

In [11]:
with open("d2l_notebooks_mapping.txt") as fp:
    mapping = eval(fp.read())
mapping

{'naive_bayes_mnist': '09',
 'image_augmentation': '19',
 'finetune_cnn': '19',
 'skipgram': '20',
 'nmt_attention': '15',
 'resnet': '14',
 'lenet': '14',
 'batchnorm': '14',
 'nmt': '15',
 'multi_head_attention': '15',
 'gru': '15',
 'cnn1d_sentiment': '15',
 'entailment_attention_mlp': '15',
 'conv2d': '14',
 'attention': '15',
 'rnn': '15',
 'densenet': '14',
 'positional_encoding': '15',
 'transformers': '15',
 'bert': '15',
 'rnn_sentiment': '15',
 'word_analogies': '20',
 'multi_gpu_training': '13',
 'lstm': '15'}

In [12]:
notebooks = glob("../notebooks/book1/*/*.ipynb")
torch_version, jax_version = {}, {}
for notebook in notebooks:
    if "_torch" in notebook:
        torch_version[notebook.split("/")[-1].split(".")[0][:-6]] = notebook.split("/")[-2]
    if "_jax" in notebook:
        jax_version[notebook.split("/")[-1].split(".")[0][:-4]] = notebook.split("/")[-2]
len(torch_version), len(jax_version)

(28, 30)

In [13]:
print("JAX - Torch:", jax_version.keys() - torch_version.keys())
print("Torch - JAX:", torch_version.keys() - jax_version.keys())

JAX - Torch: {'laplace_approx_beta_binom', 'activation_fun_deriv'}
Torch - JAX: set()


In [18]:
relative_path = "../notebooks/book1"
switch = {"torch": "jax", "jax": "torch"}


def add_pointers_in_notebooks(notebook_without_extension, chapter_no, framework, insert=True):
    path = os.path.join(relative_path, chapter_no, notebook + f"_{framework}.ipynb")
    nb = nbformat.read(path, as_version=4)
    url = make_url_from_chapter_no_and_script_name(chapter_no, notebook + f"_{switch[framework]}.ipynb")

    if is_dead_url(colab_to_githubraw_url(url)) == 1:
        print(f"{url} is dead")
    sentence = f"Please find {switch[framework]} implementation of this notebook here: {url}"

    if insert:
        nb.cells.insert(0, nbformat.v4.new_markdown_cell(sentence))
    else:
        nb.cells[0]["source"] = sentence
    nbformat.write(nb, path)

In [19]:
for notebook in torch_version:
    if notebook in mapping:
        continue
    print(notebook)
    add_pointers_in_notebooks(notebook, torch_version[notebook], "torch")
    add_pointers_in_notebooks(notebook, torch_version[notebook], "jax")

emnist_viz
https://colab.research.google.com/github/probml/pyprobml/blob/master/notebooks/book1/01/emnist_viz_torch.ipynb is dead
text_preproc
https://colab.research.google.com/github/probml/pyprobml/blob/master/notebooks/book1/01/text_preproc_torch.ipynb is dead
transposed_conv
https://colab.research.google.com/github/probml/pyprobml/blob/master/notebooks/book1/14/transposed_conv_torch.ipynb is dead
layer_norm
https://colab.research.google.com/github/probml/pyprobml/blob/master/notebooks/book1/14/layer_norm_torch.ipynb is dead


### Move d2l-notebooks

In [33]:
source_path = "../../probml-notebooks/notebooks-d2l"
dest_path = "../notebooks/book1/"

In [34]:
failed = []
for notebook in mapping:
    source_notebook_path = os.path.join(source_path, notebook + "_torch.ipynb")
    dest_notebook_path = os.path.join(dest_path, mapping[notebook], notebook + "_torch.ipynb")
    if os.system(f"cp {source_notebook_path} {dest_notebook_path}") != 0:
        failed.append([source_notebook_path, dest_notebook_path])
failed

cp: cannot stat '../../probml-notebooks/notebooks-d2l/naive_bayes_mnist_torch.ipynb': No such file or directory
cp: cannot stat '../../probml-notebooks/notebooks-d2l/multi_head_attention_torch.ipynb': No such file or directory
cp: cannot stat '../../probml-notebooks/notebooks-d2l/positional_encoding_torch.ipynb': No such file or directory


[['../../probml-notebooks/notebooks-d2l/naive_bayes_mnist_torch.ipynb',
  '../notebooks/book1/09/naive_bayes_mnist_torch.ipynb'],
 ['../../probml-notebooks/notebooks-d2l/multi_head_attention_torch.ipynb',
  '../notebooks/book1/15/multi_head_attention_torch.ipynb'],
 ['../../probml-notebooks/notebooks-d2l/positional_encoding_torch.ipynb',
  '../notebooks/book1/15/positional_encoding_torch.ipynb']]

In [35]:
for each in failed:
    source = each[0].replace("_torch", "")
    dest = each[1]
    os.system(f"cp {source} {dest}")